In [14]:
#程式碼取自 博碩, "Python 機器學習", ISBN 978-986-434-140-5, by Sebastian Raschka, 為練習Python函式庫用途
#僅供履歷作品集使用
import os
import struct
import numpy as np

def load_mnist(path, kind='train'):
    labels_path = os.path.join(path, '%s-labels.idx1-ubyte' %kind)
    images_path = os.path.join(path, '%s-images.idx3-ubyte' %kind)
    
    with open (labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II', lbpath.read(8))
        labels = np.fromfile(lbpath, dtype=np.uint8)
        
    with open (images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack('>IIII', imgpath.read(16))
        images = np.fromfile(imgpath, dtype=np.uint8).reshape(len(labels), 784)
        
    return images, labels

In [15]:
X_train, y_train = load_mnist(r'C:\Users\Acer\Desktop', kind='train')
X_test, y_test = load_mnist(r'C:\Users\Acer\Desktop', kind='t10k')
print('Rows: %d, columns: %d' % (X_train.shape[0], X_train.shape[1]))
print('Rows: %d, columns: %d' % (X_test.shape[0], X_test.shape[1]))

Rows: 60000, columns: 784
Rows: 10000, columns: 784


In [16]:
#THEANO_FLAGS=mode=FAST_RUN,device=gpu,floatX=float32 python mnist_keras_mlp.py

In [17]:
import theano
theano.config.floatX = 'float32'
X_train = X_train.astype(theano.config.floatX)
X_test = X_test.astype(theano.config.floatX)

from keras.utils import np_utils
print('First 3 labels: ', y_train[:3])
y_train_ohe = np_utils.to_categorical(y_train)
print('\nFirst 3 labels (one-hot):\n', y_train_ohe[:3])

First 3 labels:  [5 0 4]

First 3 labels (one-hot):
 [[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]]


In [18]:
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD

np.random.seed(1)
model = Sequential()
model.add(Dense(input_dim=X_train.shape[1], output_dim=50, init='uniform', activation='tanh'))
model.add(Dense(input_dim=50,output_dim=50,init='uniform', activation='tanh'))
model.add(Dense(input_dim=50,output_dim=y_train_ohe.shape[1],init='uniform', activation='softmax'))

sgd = SGD(lr=0.001, decay=1e-7, momentum=.9)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

C:\Users\Acer\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="tanh", kernel_initializer="uniform", input_dim=784, units=50)`
  import sys
C:\Users\Acer\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="tanh", kernel_initializer="uniform", input_dim=50, units=50)`
  
C:\Users\Acer\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", kernel_initializer="uniform", input_dim=50, units=10)`
  if __name__ == '__main__':


In [19]:
model.fit(X_train, y_train_ohe, nb_epoch=50, batch_size=300, verbose=1, validation_split=0.1)

C:\Users\Acer\Anaconda2\envs\tensorflow\lib\site-packages\keras\models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 54000 samples, validate on 6000 samples
Epoch 1/50
54000/54000 [==============================] - 1s - loss: 2.2244 - val_loss: 2.1019
Epoch 2/50
54000/54000 [==============================] - 1s - loss: 1.8744 - val_loss: 1.5897
Epoch 3/50
54000/54000 [==============================] - 1s - loss: 1.3845 - val_loss: 1.1463
Epoch 4/50
54000/54000 [==============================] - 1s - loss: 1.0397 - val_loss: 0.8656
Epoch 5/50
54000/54000 [==============================] - 1s - loss: 0.8219 - val_loss: 0.6892
Epoch 6/50
54000/54000 [==============================] - 1s - loss: 0.6805 - val_loss: 0.5764
Epoch 7/50
54000/54000 [==============================] - 1s - loss: 0.5890 - val_loss: 0.4936
Epoch 8/50
54000/54000 [==============================] - 1s - loss: 0.5210 - val_loss: 0.4430
Epoch 9/50
54000/54000 [==============================] - 1s - loss: 0.4755 - val_loss: 0.4135
Epoch 10/50
54000/54000 [==============================] - 1s - loss: 0.4344 - val_loss: 0.3695


In [20]:
y_train_pred = model.predict_classes(X_train, verbose=0)
y_test_pred = model.predict_classes(X_test, verbose=0)
train_acc = np.sum(y_train == y_train_pred, axis=0) / X_train.shape[0]
test_acc = np.sum(y_test == y_test_pred, axis=0) / X_test.shape[0]

print('Training accuracy: %.2f%%' % (train_acc * 100))
print('Testing accuracy: %.2f%%' % (test_acc * 100))

Training accuracy: 94.19%
Testing accuracy: 93.29%
